In [2]:
%load_ext autoreload
%autoreload 2

# !git clone https://github.com/Flagro/ExcelTableCNN.git

# !pip install py7zr
# !pip install xlrd

In [3]:
%cd /kaggle/working/ExcelTableCNN
!git pull origin main

/kaggle/working/ExcelTableCNN
From https://github.com/Flagro/ExcelTableCNN
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
import os
from ExcelTableCNN.excel_table_cnn.train_test_helpers import DatasetLoader
from ExcelTableCNN.excel_table_cnn.train_test_helpers import MarkupLoader

dataset_loader = DatasetLoader(save_path="/kaggle/working/ExcelTableCNN/data/")
# dataset_loader.get_dataset('VEnron2')
dataset_files = dataset_loader.get_files('VEnron2')

markup_loader = MarkupLoader()
markup_files = markup_loader.get_markup("tablesense")

# different extensions fix:
dataset_files["file_name_no_ext"] = dataset_files["file_name"].apply(lambda x: os.path.splitext(x)[0])
markup_files["file_name_no_ext"] = markup_files["file_name"].apply(lambda x: os.path.splitext(x)[0])

files_df = markup_files.merge(dataset_files, how="inner", on=["file_name_no_ext", "parent_path"])
files_df = files_df.drop(columns=["file_name_x", "file_name_no_ext"])
files_df = files_df.rename(columns={"file_name_y": "file_name"})

In [5]:
training_samples = files_df[files_df["set_type"] == "training_set"].sample(30)
testing_samples = files_df[files_df["set_type"] == "testing_set"].sample(10)
files_df_sample = pd.concat([training_samples, testing_samples])
files_df_sample

,sheet_name,set_type,parent_path,table_range,file_name
1248,Deal Sheet,training_set,VEnron2/VEnron2/1264,[B13:G28],10_Hillsboro Enterprises.xls
1190,View 1,training_set,VEnron2/VEnron2/1537,"[A9:E11, A15:E19]",1_DB Development.xls
29,AGL Details,training_set,VEnron2/VEnron2/868,[A1:BL57],1_CES-AGL-Feb00.xls
620,summary,training_set,VEnron2/VEnron2/1529,[A3:BA34],1_AnnualStats.xls
329,Sheet1,training_set,VEnron2/VEnron2/603,[A10:H33],1_Fuel Inventory Report.xls
608,SUM SO2,training_set,VEnron2/VEnron2/1362,[A3:AK87],10_Pwr west P&L.xls
501,Sheet2,training_set,VEnron2/VEnron2/725,[A2:G16],1_position1a.xls
469,Sheet1,training_set,VEnron2/VEnron2/715,[A2:D43],1_Govt Affairs Support For West Desk.xls
960,Sheet1,training_set,VEnron2/VEnron2/275,[A5:F24],1_BBC Jan2000 NYSEG.xls
799,Sheet1,training_set,VEnron2/VEnron2/1260,[A4:D17],1_consumptions-Sheraton Eau Claire.xls


In [6]:
import os
import pandas as pd
from ExcelTableCNN.excel_table_cnn.train_test_helpers.cell_features import get_table_features

features_dfs = []

# Iterate over the unique pairs
for _, row in files_df_sample.iterrows():
    file_path = os.path.join(row['parent_path'], row['file_name'])
    features_df = get_table_features(os.path.join("/kaggle/working/ExcelTableCNN/data", file_path), row['sheet_name'])
    features_df["file_path"] = file_path
    features_df["sheet_name"] = row['sheet_name']
    features_df["set_type"] = row["set_type"]
    features_df["table_range"] = [row["table_range"] for _ in range(len(features_df.index))]
    features_dfs.append(features_df)
files_features_df = pd.concat(features_dfs)
files_features_df.sample(30)

,coordinate,is_empty,is_string,is_bold,is_italic,formula,is_merged,left_border,right_border,top_border,...,horizontal_alignment,left_horizontal_alignment,right_horizontal_alignment,center_horizontal_alignment,wrapped_text,indent,file_path,sheet_name,set_type,table_range
27072,GK106,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
41382,FK162,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
87797,IL343,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
176518,EE690,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
148832,CS582,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
75305,AP295,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
9485,I1054,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/598/1_NAT GAS Prices GRAPH.xls,SoCal-Perm Calc,testing_set,[A1:I195]
146966,W575,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
8903,GR35,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"
131378,AY514,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,VEnron2/VEnron2/926/1_Trigger event.xls,HCC Looper,training_set,"[A45:AE69, A12:F43, G12:I43, J12:L43]"


In [33]:
train_data = files_features_df[files_features_df["set_type"] == "training_set"].drop(columns=["set_type"])
test_data = files_features_df[files_features_df["set_type"] == "testing_set"].drop(columns=["set_type"])
train_data

,coordinate,is_empty,is_string,is_bold,is_italic,formula,is_merged,left_border,right_border,top_border,...,is_filled,horizontal_alignment,left_horizontal_alignment,right_horizontal_alignment,center_horizontal_alignment,wrapped_text,indent,file_path,sheet_name,table_range
0,A1,False,True,1,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/368/1_2002planvs.mktval2.xls,Sheet1,[A6:G35]
1,B1,False,False,1,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/368/1_2002planvs.mktval2.xls,Sheet1,[A6:G35]
2,C1,False,False,1,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/368/1_2002planvs.mktval2.xls,Sheet1,[A6:G35]
3,D1,False,False,1,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/368/1_2002planvs.mktval2.xls,Sheet1,[A6:G35]
4,E1,False,False,1,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/368/1_2002planvs.mktval2.xls,Sheet1,[A6:G35]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,D45,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/144/1_gerald.xls,.,[B7:D43]
180,A46,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/144/1_gerald.xls,.,[B7:D43]
181,B46,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/144/1_gerald.xls,.,[B7:D43]
182,C46,True,False,0,0,False,False,False,False,False,...,False,False,False,False,False,False,False,VEnron2/VEnron2/144/1_gerald.xls,.,[B7:D43]


In [63]:
import pandas as pd
import numpy as np
import re

def coordinate_to_row_col(coord):
    """Convert Excel-style coordinate to row and column index."""
    col_letters, row_num = re.match(r'([A-Z]+)([0-9]+)', coord).groups()
    col_idx = sum([(ord(char) - 64) * (26**i) for i, char in enumerate(col_letters[::-1])]) - 1
    row_idx = int(row_num) - 1
    return row_idx, col_idx

def extract_bounding_boxes(table_ranges):
    """Extract bounding boxes from table ranges."""
    bounding_boxes = []
    for table_range in table_ranges:
        start_range, end_range = table_range.split(":")
        start_row, start_col = coordinate_to_row_col(start_range)
        end_row, end_col = coordinate_to_row_col(end_range)
        bounding_boxes.append((start_row, start_col, end_row, end_col))
    return bounding_boxes

df = files_features_df.copy()  # Your dataframe

results = {}
FIXED_N, FIXED_M = 150, 50

def create_empty_tensor(C):
    return np.zeros((FIXED_N, FIXED_M, C))

# Group by each sheet
for (_, group) in df.groupby(['file_path', 'sheet_name']):
    tensors = []
    target_tensors = []  # To hold the target bounding boxes
    table_ranges = group['table_range'].iloc[0]
    C = len(group.columns) - 5 + 1
    
    bounding_boxes = extract_bounding_boxes(table_ranges)

    # Identify the starting point of data in this group
    min_row_idx = min(coordinate_to_row_col(coord)[0] for coord in group['coordinate'])
    min_col_idx = min(coordinate_to_row_col(coord)[1] for coord in group['coordinate'])

    for row_block_start in range(min_row_idx, min_row_idx + FIXED_N, FIXED_N):
        for col_block_start in range(min_col_idx, min_col_idx + FIXED_M, FIXED_M):
            tensor = create_empty_tensor(C)
            
            block_group = group[group['coordinate'].apply(lambda coord: 
                            row_block_start <= coordinate_to_row_col(coord)[0] < row_block_start + FIXED_N and 
                            col_block_start <= coordinate_to_row_col(coord)[1] < col_block_start + FIXED_M)]
            
            # Extract bounding boxes that fall within this block
            block_bboxes = [(start_row - row_block_start, start_col - col_block_start, end_row - row_block_start, end_col - col_block_start)
                            for (start_row, start_col, end_row, end_col) in bounding_boxes
                            if row_block_start <= start_row < row_block_start + FIXED_N and
                               col_block_start <= start_col < col_block_start + FIXED_M]

            for _, row in block_group.iterrows():
                row_idx, col_idx = coordinate_to_row_col(row['coordinate'])
                features = row.drop(['file_path', 'sheet_name', 'coordinate', 'table_range', 'set_type']).values

                tensor[row_idx - row_block_start, col_idx - col_block_start, :-1] = features

            tensors.append(tensor)
            target_tensors.append({"boxes": block_bboxes})

    results[(group["file_path"].iloc[0], group["sheet_name"].iloc[0])] = {"tensors": tensors, "targets": target_tensors, "set_type": group["set_type"].iloc[0]}

In [100]:
import torch

# Lists for training and testing
train_images = []
test_images = []
train_targets_list = []
test_targets_list = []

# Iterate through the results to prepare PyTorch tensors and targets
for key, value in results.items():
    for tensor, target in zip(value["tensors"], value["targets"]):
        # Convert numpy tensor to PyTorch tensor
        tensor_pt = torch.tensor(tensor, dtype=torch.float32)

        # Use all channels except the last one for the input image
        image_tensor = tensor_pt[:, :, :-1]
        image_tensor = image_tensor.permute(2, 0, 1)  # If you need to change the order of dimensions

        boxes = torch.tensor(target["boxes"], dtype=torch.float32)  # Convert bounding boxes to PyTorch tensor
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)  # Assuming each box is a "table" with label 1
        
        target_dict = {
            "boxes": boxes,
            "labels": labels
        }
        
        if value["set_type"] == "training_set":
            train_images.append(image_tensor)
            train_targets_list.append(target_dict)
        elif value["set_type"] == "testing_set":
            test_images.append(image_tensor)
            test_targets_list.append(target_dict)

In [104]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):  # Assuming 2 classes: table and background
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(17, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128 * 18 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )
        
        self.regressor = nn.Sequential(
            nn.Linear(128 * 18 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 4)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        class_scores = self.classifier(x)
        bbox = self.regressor(x)
        return class_scores, bbox

# Create the model, criterion, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion_class = nn.CrossEntropyLoss()
criterion_bbox = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, target_dict in zip(train_images, train_targets_list):
        # Skip images with no bounding boxes
        if target_dict["boxes"].shape[0] == 0:
            continue

        optimizer.zero_grad()

        images = images.unsqueeze(0).to(device)  # Add a batch dimension and send to device
        class_scores, bbox_preds = model(images)

        # Adjusting the target for classification to have batch size dimension
        labels_for_image = target_dict["labels"][0].unsqueeze(0).to(device)

        loss_class = criterion_class(class_scores, labels_for_image)

        # Here we just take the first bounding box from the target as the "representative" box
        representative_box = target_dict["boxes"][0][0].unsqueeze(0).to(device)

        loss_bbox = criterion_bbox(bbox_preds, representative_box)

        loss = loss_class + loss_bbox

        loss.backward()
        optimizer.step()

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [105]:
# Evaluation


### THIS IS JUST A TEST ON A SMALL CHUNK OF DATA

model.eval()  # Set the model to evaluation mode

total_correct = 0
total_images = 0
total_bbox_mse = 0

with torch.no_grad():  # No need to track gradients during evaluation
    for images, target_dict in zip(test_images, test_targets_list):
        if target_dict["boxes"].shape[0] == 0:
            continue

        images = images.unsqueeze(0).to(device)
        class_scores, bbox_preds = model(images)

        # Get the predicted class label
        _, predicted_class = torch.max(class_scores, 1)

        # Compute number of correct predictions
        correct_predictions = (predicted_class == target_dict["labels"][0].to(device)).sum().item()
        total_correct += correct_predictions
        total_images += images.size(0)

        # Compute MSE for bounding box regression
        representative_box = target_dict["boxes"][0][0].unsqueeze(0).to(device)
        mse = criterion_bbox(bbox_preds, representative_box)
        total_bbox_mse += mse.item()

# Compute accuracy and average MSE
classification_accuracy = total_correct / total_images
average_bbox_mse = total_bbox_mse / len(test_images)

print(f"Classification Accuracy: {classification_accuracy:.4f}")
print(f"Average BBox MSE: {average_bbox_mse:.4f}")

Classification Accuracy: 1.0000
Average BBox MSE: 24.0678
